In [9]:
import mpld3
import csv
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np
from scipy import signal
import warnings

warnings.simplefilter('ignore')
FILE_NAME = 'atm_daily.txt'

In [10]:
def get_data():
    atm_data = pd.read_csv(FILE_NAME,sep='|')
    return atm_data

def smoothTriangle(data, degree):
    triangle=np.concatenate((np.arange(degree + 1), np.arange(degree)[::-1])) # up then down
    smoothed=[]

    for i in range(degree, len(data) - degree * 2):
        point=data[i:i + len(triangle)] * triangle
        smoothed.append(np.sum(point)/np.sum(triangle))
    # Handle boundaries
    smoothed=[smoothed[0]]*int(degree + degree/2) + smoothed
    while len(smoothed) < len(data):
        smoothed.append(smoothed[-1])
    return smoothed

In [11]:
atm_df = get_data()
atm_df.head()

one_atm_df = atm_df.query('atm_id==1')
x = one_atm_df['trans_date']
y = one_atm_df['bal']

In [12]:
fig = go.Figure(data=go.Line(x=x,y=y))
fig.show()

In [13]:
fig = go.Figure(data=go.Line(x=x,y=y))
fig.add_trace(go.Scatter(
    x=x,
    y=signal.savgol_filter(y, 
                           53, # window size used for filtering 
                           3), # order of fitted polynomial
    mode='markers',
    marker=dict(
        size=6,
        color='mediumpurple',
        symbol='triangle-up'
    ),
    name='Savitzky-Golay'
))


In [14]:
fig = go.Figure(data=go.Line(x=x,y=y))
fig.add_trace(go.Scatter(
    x=x,
    y=smoothTriangle(y, 10),  # setting degree to 10
    mode='markers',
    marker=dict(
        size=6,
        
        color='red',
        symbol='triangle-up'
    ),
    name='Moving Triangle - Degree 10'
))

In [17]:
one_atm_df.trans_date = pd.to_datetime(one_atm_df['trans_date'])
resample = one_atm_df.resample('M')
monthly_mean = resample.mean()
fig = go.Figure(data=go.Line(x=x,y=monthly_mean))
fig.add_trace(go.Scatter(
    x=x,
    y=smoothTriangle(y, 10),  # setting degree to 10
    mode='markers',
    marker=dict(
        size=6,
        color='red',
        symbol='triangle-up'
    ),
    name='Moving Triangle - Degree 10'
))

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Int64Index'

In [18]:
from statsmodels.tsa.seasonal import seasonal_decompose

result = seasonal_decompose(one_atm_df, model='multiplicative')
result.plot()
pyplot.show()

AttributeError: 'Int64Index' object has no attribute 'inferred_freq'